In [1]:
import os
import h5py
import datetime
import numpy as np
from tqdm import tqdm
from provabgs import infer as Infer

In [2]:
# get healpixels and target ids of rerun posteriors
dat_dir = '/global/cfs/cdirs/desi/users/chahah/provabgs/svda/'

In [4]:
for target in ['BGS_BRIGHT', 'BGS_FAINT']: 
    hpixs, tids = np.loadtxt(os.path.join(dat_dir, f'sv3-bright-{target}.flagged.dat'),
                             dtype=int, unpack=True, usecols=[0,1])
    uhpixs = np.unique(hpixs)

    missed_hpix, missed_tids = [], []
    for hpix in tqdm(uhpixs):
        tids_redone = tids[hpixs == hpix]

        fhpix = os.path.join(dat_dir, f'provabgs-sv3-bright-{hpix}.{target}.hdf5')

        #if os.path.isfile(fhpix) and datetime.datetime.fromtimestamp(os.path.getmtime(fhpix)).month == 12:
        #    if datetime.datetime.fromtimestamp(os.path.getmtime(fhpix)).day >= 22:
        #        print('already updated %s' % fhpix)
        #        continue        
        #print('updating %i posteriors in healpix %i' % (len(tids_redone), hpix))

        # read hpix file
        with h5py.File(fhpix, 'r+') as fhpix:
            for tid in tids_redone:
                is_tid = (fhpix['targetid'][...] == tid)
                if np.sum(is_tid) == 0: continue # hmm, it's not in the original file for whatever reason
                i_tid = np.arange(len(fhpix['targetid'][...]))[is_tid][0]

                # read updated posterior file
                fmcmc = os.path.join(dat_dir, 'healpix', str(hpix), 'provabgs.%i.hdf5' % tid)

                if not os.path.isfile(fmcmc):
                    print(fmcmc)
                    print('\t healpix %i targetid %i missing' % (hpix, tid))
                    missed_hpix.append(hpix)
                    missed_tids.append(tid)
                    continue

                post_i = Infer.PostOut()
                post_i.read(fmcmc)

                # update value
                fhpix['samples'][i_tid]     = post_i.samples
                fhpix['log_prob'][i_tid]    = post_i.log_prob
                fhpix['redshift'][i_tid]    = post_i.redshift
                fhpix['wavelength_obs'][i_tid]  = post_i.wavelength_obs
                fhpix['flux_spec_obs'][i_tid]   = post_i.flux_spec_obs
                fhpix['ivar_spec_obs'][i_tid]   = post_i.ivar_spec_obs
                fhpix['flux_photo_obs'][i_tid]  = post_i.flux_photo_obs
                fhpix['ivar_photo_obs'][i_tid]  = post_i.ivar_photo_obs

                fhpix['flux_spec_model'][i_tid]   = post_i.flux_spec_model
                fhpix['flux_photo_model'][i_tid]  = post_i.flux_spec_model # this is a bug but it will blow up otherwise
    print(missed_hpix)
    print(missed_tids)

  9%|▉         | 29/324 [03:30<16:47,  3.42s/it]  

/global/cfs/cdirs/desi/users/chahah/provabgs/svda/healpix/9819/provabgs.39633127684967516.hdf5
	 healpix 9819 targetid 39633127684967516 missing


 11%|█         | 36/324 [04:09<22:58,  4.79s/it]

/global/cfs/cdirs/desi/users/chahah/provabgs/svda/healpix/9834/provabgs.39633118675601593.hdf5
	 healpix 9834 targetid 39633118675601593 missing


 30%|███       | 98/324 [12:17<41:02, 10.90s/it]

...

/global/cfs/cdirs/desi/users/chahah/provabgs/svda/healpix/15354/provabgs.39633425283418106.hdf5
	 healpix 15354 targetid 39633425283418106 missing


 84%|████████▎ | 271/324 [34:59<08:17,  9.38s/it]

/global/cfs/cdirs/desi/users/chahah/provabgs/svda/healpix/26285/provabgs.39627824046278274.hdf5
	 healpix 26285 targetid 39627824046278274 missing


100%|██████████| 324/324 [41:21<00:00,  7.66s/it]

[9819, 9834, 10379, 10404, 10513, 15354, 26285]
[39633127684967516, 39633118675601593, 39628438629254793, 39628470447245344, 39632966409783953, 39633425283418106, 39627824046278274]
